In [1]:
from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random
import torch.nn.functional as F
from datetime import datetime
import matplotlib.pyplot as plt
import cv2


In [2]:
GAMMA=0.99
BATCH_SIZE=512
BUFFER_SIZE=50000
MIN_REPLAY_SIZE=10000
EPSILON_START=1.0
EPSILON_END=0.02
EPSILON_DECAY=150000
MAX_REWARD_REACH = -100
TARGET_UPDATE_FREQ = 1000
LR =1e-5
ENV_Name='Acrobot-v1'
TOTAl_episode =250000
save_name='mean'

In [3]:
torch.manual_seed(2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
env = gym.make(ENV_Name)
env.seed(0)
env.env.book_or_nips = 'nips'

/home/ragu_robo/.local/lib/python3.8/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/ragu_robo/.local/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/ragu_robo/.local/lib/python3.8/site-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [4]:
class DuelingDQN_Mean(nn.Module):
    def __init__(self, env):
        super(DuelingDQN_Mean, self).__init__()

        in_features = int(np.prod(env.observation_space.shape))

        self.fc1 = nn.Linear(in_features, 128)
        self.fc2 = nn.Linear(128, 128)

        self.fc3 = nn.Linear(128, 128)
        '''Value stream'''
        self.value_stream = nn.Linear(128, 1)

        '''Advantage stream'''
        self.advantage_stream = nn.Linear(128, env.action_space.n)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))


        value = self.value_stream(x)
        advantage = self.advantage_stream(x)

        '''Combining value and advantage streams'''
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        # q_values = value + (advantage - advantage.max(dim=1, keepdim=True)[0])

        return q_values
    
    def act(self, obs):
        obs_t = torch.as_tensor(obs, dtype=torch.float32)
        q_values = self(obs_t.unsqueeze(0))
        max_q_index = torch.argmax(q_values, dim=1)[0]
        action = max_q_index.cpu().detach().item()
        return action

In [5]:
# import gym
# import cv2

# # Define your environment
# env_name = 'CartPole-v1'
# env = gym.make(env_name)

# # Reset the environment
# observation = env.reset()

# # Define video writer
# video_path = 'output.avi'
# fps = 30.0
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(video_path, fourcc, fps, (600, 400))  # Change resolution as needed

# done = False
# for c in range(1000):
#     # Render the environment
#     frame = env.render(mode='rgb_array')

#     # Convert the frame to BGR for OpenCV
#     frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

#     # Write frame to the video
#     out.write(frame_bgr)

#     # Take an action
#     action = env.action_space.sample()  # Random action for demonstration
#     observation, reward, done, info = env.step(action)

# # Release video writer and close environment
# out.release()
# env.close()

In [6]:
path = f'./All_ckpt/iteration_4_Acrobot-v1_Q{save_name}_best_checkpoint__.pth'
online_net = DuelingDQN_Mean(env).to(device)
pretrained_statedict = torch.load(path)
online_net.load_state_dict(pretrained_statedict["state_dict"])



obs = env.reset()
obs = torch.as_tensor(obs, dtype=torch.float32).to(device)
action = online_net.act(obs)

In [11]:
# Define video writer
video_path = f'Q1Acrobot{save_name}.mp4'
fps = 30.0
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use mp4v codec for mp4 files
out = cv2.VideoWriter(video_path, fourcc, fps, (500, 500))  # Change resolution as needed

state = env.reset() 
for c in range(300):
    frame = env.render(mode='rgb_array')
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    print(frame_bgr.shape)
    out.write(frame_bgr)

#     # Take an action
#     env.render()
    # Random action for demonstration
    state = torch.as_tensor(state, dtype=torch.float32).to(device)
    action = online_net.act(state)
    state, reward, done, info = env.step(action)
    
    if done:
        state = env.reset() 

        
    

#     env.render()

out.release()
env.close()

(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 